In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os
import time
import concurrent.futures
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

/Users/leilayfarsani/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()
api_key = os.getenv('IATI_API_KEY')

if not api_key:
    raise ValueError("API key not found. Please make sure it is set in the .env file or update it if necessary.")

Activity/apache/select

In [ ]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def fetch_page(start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity"))',
        #'fl': 'iati_identifier,title_narrative,description_narrative,sector_code,activity_date_iso_date,activity_date_type,recipient_country_code',
        'fq': 'activity_date_type:2 AND activity_date_iso_date:[2021-01-01T00:00:00Z TO *]',  
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    for attempt in range(5):  
        try:
            response = requests_retry_session().get(base_url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            docs = response.json()['response']['docs']
            return docs, len(docs)
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                wait = 2 ** attempt  # exponential backoff
                print(f"Rate limit hit. Waiting for {wait} seconds.")
                time.sleep(wait)
            else:
                print(f"HTTP error occurred: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
        
    print(f"Failed to fetch data for start={start} after 5 attempts")
    return [], 0

def get_total_results():
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity"))',
        'rows': 0
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests_retry_session().get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()['response']['numFound']
    else:
        print(f"Error: {response.status_code}")
        return 0

base_url = "https://api.iatistandard.org/datastore/activity/select"
total_results = get_total_results() 
all_activities = []
page_sizes = []

print(f"Total results to fetch: {total_results}")

max_empty_pages = 5  
empty_page_count = 0

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    futures = [executor.submit(fetch_page, i) for i in range(0, total_results, 1000)]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        docs, size = future.result()
        all_activities.extend(docs)
        page_sizes.append(size)
        print(f"Fetched page {i+1}/{len(futures)} with {size} documents")
        
        if size == 0:
            empty_page_count += 1
            if empty_page_count >= max_empty_pages:
                print(f"Stopped fetching after {max_empty_pages} consecutive empty pages")
                break
        else:
            empty_page_count = 0

        time.sleep(1)  # Adding a small delay between requests

print(f"Page sizes: {page_sizes}")
print(f"Sum of page sizes: {sum(page_sizes)}")
print(f"Total activities fetched: {len(all_activities)}")

df_activities = pd.DataFrame(all_activities)

def clean_list_field(field):
    return field[0] if isinstance(field, list) and len(field) > 0 else field

for col in ['sector_code', 'title_narrative', 'description_narrative', 'recipient_country_code']:
    df_activities[col] = df_activities[col].apply(clean_list_field)

df_activities['start_date'] = pd.to_datetime(df_activities['activity_date_iso_date'].apply(clean_list_field), errors='coerce')

print(df_activities.head())
print(f"Shape of DataFrame: {df_activities.shape}")


Data Validation:
Performing some additional checks on the data to ensure its quality and completeness.

In [7]:
print(df_activities['sector_code'].value_counts())
print(df_activities['recipient_country_code'].value_counts())
print(df_activities['start_date'].min(), df_activities['start_date'].max())

sector_code
31120       640
6           622
72010       578
3           552
311         403
           ... 
31281         1
34-01-01      1
34-01-07      1
11430         1
BH            1
Name: count, Length: 253, dtype: int64
recipient_country_code
AF    430
ET    422
SS    337
SO    331
YE    272
     ... 
GZ      1
CW      1
CA      1
ES      1
NR      1
Name: count, Length: 174, dtype: int64
1979-01-01 00:00:00+00:00 2028-12-31 00:00:00+00:00


Date Analysis



In [8]:
early_dates = df_activities[df_activities['start_date'] < '2021-01-01']
print(f"Number of activities with start date before 2021-01-01: {len(early_dates)}")
print(early_dates[['iati_identifier', 'start_date', 'activity_date_iso_date']].head())

Number of activities with start date before 2021-01-01: 4415
                               iati_identifier                start_date  \
2                          GB-CHC-220949-P8501 2020-06-20 00:00:00+00:00   
10                         XM-DAC-41301-663721 2020-03-01 00:00:00+00:00   
12                     XM-DAC-3-1-264893-32579 2018-01-29 00:00:00+00:00   
14            XI-IATI-EC_INTPA-2020-PCC-412348 2020-11-27 00:00:00+00:00   
25  XM-OCHA-CBPF-AFG-19/3481/RA4/FSAC/UN/14864 2020-01-01 00:00:00+00:00   

                               activity_date_iso_date  
2        [2020-06-20T00:00:00Z, 2021-06-14T00:00:00Z]  
10       [2020-03-01T00:00:00Z, 2023-02-28T00:00:00Z]  
12  [2018-01-29T00:00:00Z, 2020-09-23T00:00:00Z, 2...  
14  [2020-11-27T00:00:00Z, 2020-11-27T00:00:00Z, 2...  
25  [2020-01-01T00:00:00Z, 2020-01-01T00:00:00Z, 2...  


In [9]:
def parse_dates(date_list):
    return [date for date in date_list if date.startswith('2')]  

early_dates['parsed_dates'] = early_dates['activity_date_iso_date'].apply(parse_dates)
print(early_dates[['iati_identifier', 'start_date', 'parsed_dates']].head())

                               iati_identifier                start_date  \
2                          GB-CHC-220949-P8501 2020-06-20 00:00:00+00:00   
10                         XM-DAC-41301-663721 2020-03-01 00:00:00+00:00   
12                     XM-DAC-3-1-264893-32579 2018-01-29 00:00:00+00:00   
14            XI-IATI-EC_INTPA-2020-PCC-412348 2020-11-27 00:00:00+00:00   
25  XM-OCHA-CBPF-AFG-19/3481/RA4/FSAC/UN/14864 2020-01-01 00:00:00+00:00   

                                         parsed_dates  
2        [2020-06-20T00:00:00Z, 2021-06-14T00:00:00Z]  
10       [2020-03-01T00:00:00Z, 2023-02-28T00:00:00Z]  
12  [2018-01-29T00:00:00Z, 2020-09-23T00:00:00Z, 2...  
14  [2020-11-27T00:00:00Z, 2020-11-27T00:00:00Z, 2...  
25  [2020-01-01T00:00:00Z, 2020-01-01T00:00:00Z, 2...  


/var/folders/1y/bbgshr8n1m3fx_npbqvt9_w00000gn/T/ipykernel_57858/2312078001.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  early_dates['parsed_dates'] = early_dates['activity_date_iso_date'].apply(parse_dates)


In [10]:
def get_earliest_date(date_list):
    return min(parse_dates(date_list), default=None)

df_activities['earliest_date'] = df_activities['activity_date_iso_date'].apply(get_earliest_date)
df_activities['earliest_date'] = pd.to_datetime(df_activities['earliest_date'])

print(df_activities[['start_date', 'earliest_date']].head())
print(f"Number of activities where earliest_date != start_date: {(df_activities['earliest_date'] != df_activities['start_date']).sum()}")

                 start_date             earliest_date
0 2027-02-01 00:00:00+00:00 2022-06-21 00:00:00+00:00
1 2022-05-08 00:00:00+00:00 2022-05-08 00:00:00+00:00
2 2020-06-20 00:00:00+00:00 2020-06-20 00:00:00+00:00
3 2024-02-05 00:00:00+00:00 2024-02-05 00:00:00+00:00
4 2023-12-01 00:00:00+00:00 2023-12-01 00:00:00+00:00
Number of activities where earliest_date != start_date: 721


Activity/XML

In [11]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def fetch_page_xml(start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity"))',
        'fq': 'activity_date_iso_date:[2021-01-01T00:00:00Z TO *]',
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    response = requests_retry_session().get(xml_base_url, headers=headers, params=params, timeout=30)
    if response.status_code == 200:
        return response.content  
    else:
        response.raise_for_status()

def parse_xml_data(xml_data):
    root = ET.fromstring(xml_data)
    activities = []
    for activity in root.findall('.//iati-activity'):
        iati_id = activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None
        title = activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None
        description = activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None
        
        sector_element = activity.find('.//sector/code')
        sector = sector_element.attrib.get('code') if sector_element is not None else None
        
        date_element = activity.find('.//activity-date[@type="2"]')  # Type "2" is the actual start date
        date = date_element.attrib.get('iso-date') if date_element is not None else None
        
        country_element = activity.find('.//recipient-country')
        country = country_element.attrib.get('code') if country_element is not None else None
        
        activities.append({
            'iati_identifier': iati_id,
            'title_narrative': title,
            'description_narrative': description,
            'sector_code': sector,
            'activity_date_iso_date': date,
            'recipient_country_code': country
        })
    return activities

def fetch_all_pages():
    all_activities = []
    start = 0
    while True:
        xml_data = fetch_page_xml(start=start)
        activities = parse_xml_data(xml_data)
        if not activities:  
            break
        all_activities.extend(activities)
        start += 1000  
    return all_activities

xml_base_url = "https://api.iatistandard.org/datastore/activity/iati"  # Updated URL for XML

all_activities = fetch_all_pages()

df_activity_xml = pd.DataFrame(all_activities)
print(f"Total activities fetched: {len(df_activity_xml)}")
print(df_activity_xml.head())


Total activities fetched: 11823
                  iati_identifier  \
0                NZ-1-ACT-0103019   
1  NL-KVK-40409352-PRJ13-108-0004   
2             GB-CHC-220949-P8501   
3             XM-DAC-41301-745086   
4                 SE-0-SE-6-16450   

                                     title_narrative  \
0                2022 Horn of Africa Food Insecurity   
1                      2022 Food insecurity EA Niger   
2                    Mauritania Food Insecurity 2020   
3  Supporting Food Security Preparedness and Resi...   
4   ACF/NRC Food and nutrition security Cabo Delgado   

                               description_narrative sector_code  \
0  Humanitarian contributions to support the inte...        None   
1  Niger is currently being affected by the worst...        None   
2  Mauritania is currently facing a difficult foo...        None   
3  To achieve higher levels of food security in t...        None   
4  The contribution proposed will span over three...        None   


Activity/XML considering SDG tag code 2

OR sdg tag2

In [13]:
def requests_retry_session(session=None, retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504)):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def fetch_page_xml(session, start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity")) OR tag_vocabulary:2',
        'fq': 'activity_date_iso_date:[2021-01-01T00:00:00Z TO *]',
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    response = session.get(xml_base_url, headers=headers, params=params, timeout=30)
    if response.status_code == 200:
        return response.content  
    else:
        response.raise_for_status()

def parse_xml_data(xml_data):
    root = ET.fromstring(xml_data)
    activities = []
    for activity in root.findall('.//iati-activity'):
        iati_id = activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None
        title = activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None
        description = activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None
        
        sector_element = activity.find('.//sector/code')
        sector = sector_element.attrib.get('code') if sector_element is not None else None
        
        date_element = activity.find('.//activity-date[@type="2"]')  
        date = date_element.attrib.get('iso-date') if date_element is not None else None
        
        country_element = activity.find('.//recipient-country')
        country = country_element.attrib.get('code') if country_element is not None else None
        
        # Extract SDG tags
        sdg_tags = activity.findall(".//tag[@vocabulary='2']")
        sdg_codes = [tag.get('code') for tag in sdg_tags]
        
        activities.append({
            'iati_identifier': iati_id,
            'title_narrative': title,
            'description_narrative': description,
            'sector_code': sector,
            'activity_date_iso_date': date,
            'recipient_country_code': country,
            'sdg_tags': sdg_codes  
        })
    return activities

def fetch_all_records():
    session = requests_retry_session()  
    all_activities = []
    start = 0
    
    while True:
        xml_data = fetch_page_xml(session, start)
        activities = parse_xml_data(xml_data)
        if not activities:  
            break
        all_activities.extend(activities)
        start += 1000  
    
    return all_activities


xml_base_url = "https://api.iatistandard.org/datastore/activity/iati"  

all_activities = fetch_all_records()
df_activity_xml_sdg = pd.DataFrame(all_activities)
print(f"Total activities fetched: {len(df_activity_xml_sdg )}")


Total activities fetched: 64729


And sdg tag2

In [15]:
def requests_retry_session(session=None, retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504)):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def fetch_page_xml(session, start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity")) AND tag_vocabulary:2',
        'fq': 'activity_date_iso_date:[2021-01-01T00:00:00Z TO *]',
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    response = session.get(xml_base_url, headers=headers, params=params, timeout=30)
    if response.status_code == 200:
        return response.content  
    else:
        response.raise_for_status()

def parse_xml_data(xml_data):
    root = ET.fromstring(xml_data)
    activities = []
    for activity in root.findall('.//iati-activity'):
        iati_id = activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None
        title = activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None
        description = activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None
        
        sector_element = activity.find('.//sector/code')
        sector = sector_element.attrib.get('code') if sector_element is not None else None
        
        date_element = activity.find('.//activity-date[@type="2"]')  
        date = date_element.attrib.get('iso-date') if date_element is not None else None
        
        country_element = activity.find('.//recipient-country')
        country = country_element.attrib.get('code') if country_element is not None else None
        
        # Extract SDG tags
        sdg_tags = activity.findall(".//tag[@vocabulary='2']")
        sdg_codes = [tag.get('code') for tag in sdg_tags]
        
        activities.append({
            'iati_identifier': iati_id,
            'title_narrative': title,
            'description_narrative': description,
            'sector_code': sector,
            'activity_date_iso_date': date,
            'recipient_country_code': country,
            'sdg_tags': sdg_codes  
        })
    return activities

def fetch_all_records():
    session = requests_retry_session()  
    all_activities = []
    start = 0
    
    while True:
        xml_data = fetch_page_xml(session, start)
        activities = parse_xml_data(xml_data)
        if not activities:  
            break
        all_activities.extend(activities)
        start += 1000  
    
    return all_activities


xml_base_url = "https://api.iatistandard.org/datastore/activity/iati"  

all_activities = fetch_all_records()
df_activity_xml_and_sdg = pd.DataFrame(all_activities)



### Transactions/Apache-Solr default

Note: Transactions in XML format is not accessible from Datastore API

In [17]:
def requests_retry_session(retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504), session=None):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def fetch_page(start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (description_narrative:("food security" OR "food insecurity"))',
        'fl': 'iati_identifier,transaction_value,transaction_date_iso_date,sector_code,recipient_country_code',
        'fq': 'transaction_transaction_date_iso_date:[2021-01-01T00:00:00Z TO *]',  
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    for attempt in range(5):  
        try:
            response = requests_retry_session().get(base_url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            return response.json()['response']['docs']
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                wait = 2 ** attempt  # exponential backoff
                print(f"Rate limit hit. Waiting for {wait} seconds.")
                time.sleep(wait)
            else:
                print(f"HTTP error occurred: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
        
    print(f"Failed to fetch data for start={start} after 5 attempts")
    return []

def get_total_results():
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (description_narrative:("food security" OR "food insecurity"))',
        'rows': 0
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests_retry_session().get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()['response']['numFound']
    else:
        print(f"Error: {response.status_code}")
        return 0

# Updated base URL for transaction collection
base_url = "https://api.iatistandard.org/datastore/transaction/select"

total_results = get_total_results()
all_transactions = []

print(f"Total results to fetch: {total_results}")

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor: 
    futures = [executor.submit(fetch_page, i) for i in range(0, total_results, 1000)]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        all_transactions.extend(future.result())
        print(f"Fetched page {i+1}/{len(futures)}")
        time.sleep(1) 

df_transactions = pd.DataFrame(all_transactions)

Total results to fetch: 257707
Fetched page 1/258
Fetched page 2/258
Fetched page 3/258
Fetched page 4/258
Fetched page 5/258
Fetched page 6/258
Fetched page 7/258
Fetched page 8/258
Fetched page 9/258
Fetched page 10/258
Fetched page 11/258
Fetched page 12/258
Fetched page 13/258
Fetched page 14/258
Fetched page 15/258
Fetched page 16/258
Fetched page 17/258
Fetched page 18/258
Fetched page 19/258
Fetched page 20/258
Fetched page 21/258
Fetched page 22/258
Fetched page 23/258
Fetched page 24/258
Fetched page 25/258
Fetched page 26/258
Fetched page 27/258
Fetched page 28/258
Fetched page 29/258
Fetched page 30/258
Fetched page 31/258
Fetched page 32/258
Fetched page 33/258
Fetched page 34/258
Fetched page 35/258
Fetched page 36/258
Fetched page 37/258
Fetched page 38/258
Fetched page 39/258
Fetched page 40/258
Fetched page 41/258
Fetched page 42/258
Fetched page 43/258
Fetched page 44/258
Fetched page 45/258
Fetched page 46/258
Fetched page 47/258
Fetched page 48/258
Fetched page 49/25